# 概述
区域协调，路网为4*4矩阵，索引$i, j\in [0, 3]$    
路网所有交叉口采用对称放行    
其中干道的协调方法使用改进型数解法      
交叉口距离用二维向量表示为$(d_{(i-1, j)\to(i, j)}, d_{(i, j-1)\to(i, j)})$

In [12]:
import pandas as pd
import numpy as np
import math

# parameters

# dis_matrix: each element is a pair of distances
# first is the dis from current to left, second is the dis from current to up
dis_matrix = [[(0, 0), (250, 0), (540, 0), (180, 0)], 
                [(0, 300), (250, 390), (300, 420), (200, 400)], 
                [(0, 280), (230, 210), (340, 410), (256, 128)], 
                [(0, 320), (170, 200), (340, 320), (540, 450)]]

c_min = 40
c_max = 120
v = 12    # unit: m/s
step = 1

In [13]:
idea_dis_min = math.ceil(0.5 * v * c_min)
idea_dis_max = math.floor(0.5 * v * c_max)
print("idea distance in: {} to {}".format(idea_dis_min, idea_dis_max))

idea distance in: 240 to 720


In [14]:
class LineCoordination:
    def __init__(self, dis_list):

        self.dis_list = dis_list
        self.index_list = [i for i in range(len(dis_list))]
        cols = ['idea_dis'] + self.index_list + ['b', 'b/a', 'b_index']
        self.dis_diff = pd.DataFrame(columns=cols)

        self.dis_pre = [0]
        for i in range(1, len(self.dis_list)):
            self.dis_pre.append(self.dis_pre[-1] + dis_list[i])
        #print(dis_pre)

        self.optimal_cycle = 0
        self.dis_diff_opt = None

    def findOptimal(self):

        self.dis_diff.drop(index=self.dis_diff.index)
        self.dis_diff_opt = None

        for i in range(idea_dis_min, idea_dis_max+step, step):
            tmp = []
            for j in self.dis_pre:
                tmp.append(j % i)
        
            #print(tmp)

            tmp_sorted = sorted(tmp)
            tmp_sorted.append(i)
            #print(tmp_sorted)

            tmp_diff = []
            for j in range(1, len(tmp_sorted)):
                tmp_diff.append(tmp_sorted[j] - tmp_sorted[j-1])
            #tmp_diff.append(tmp_sorted[0] - tmp_sorted[-1])
            #print(tmp_diff)

            diff_max = max(tmp_diff)
            diff_max_id = tmp_diff.index(diff_max)
            #print(diff_max)

            row = [i] + tmp + [diff_max, (diff_max / i), diff_max_id]
            #print(row)

            self.dis_diff.loc[len(self.dis_diff.index)] = row 

        dis_diff_max = self.dis_diff[self.dis_diff['b/a'] == self.dis_diff['b/a'].max()]
        dis_diff_max = dis_diff_max.reset_index(drop=True)
        self.dis_diff_opt = dis_diff_max.loc[0]

        dis_opt = dis_diff_max.at[0, 'idea_dis']
        self.optimal_cycle = 2 * dis_opt / v

        return self.optimal_cycle, self.dis_diff, self.dis_diff_opt

    def getOptimalInfo(self, ratio):

        if self.dis_diff_opt.empty:
            print("Error! optimal has not been found. Please run findOptimal. ")
            return None, None

        ideas_count = int(np.ceil(self.dis_pre[-1] / self.dis_diff_opt['idea_dis']) + 1)

        id_reals_to_ideas = []    # the id of ideas matched to reals
        pos_reals_to_ideas = []    # 0 means real is on the left, 1 means right
        dis_reals_to_ideas = []

        list_dis_diff_opt = self.dis_diff_opt[1:-3]
        list_dis_diff_opt = list(list_dis_diff_opt.values)
        list_dis_diff_opt.sort()

        x = (self.dis_diff_opt['idea_dis'] - self.dis_diff_opt['b']) / 2
        y = list_dis_diff_opt[int(self.dis_diff_opt['b_index'])]
        offset = x - y    # offset for dis_pre
        dis_pre_offseted = [i + offset for i in self.dis_pre]
        #print("offseted: {}".format(dis_pre_offseted))

        for i in range(len(dis_pre_offseted)):
            dis_real_to_ideas = []
            dis_abs = []

            for j in range(ideas_count):
                tmp = dis_pre_offseted[i] - self.dis_diff_opt['idea_dis']*j
                dis_real_to_ideas.append(tmp)
                dis_abs.append(np.abs(tmp))
            #print(dis_real_to_ideas)

            dis_min = np.min(dis_abs)
            dis_min_index = dis_abs.index(dis_min)
            #print("id: {}, val: {}".format(dis_min_index, dis_min))

            id_reals_to_ideas.append(dis_min_index)
            dis_reals_to_ideas.append(dis_real_to_ideas[dis_min_index])

            if dis_real_to_ideas[dis_min_index] < 0:
                pos_reals_to_ideas.append(0)
            else:
                pos_reals_to_ideas.append(1)

        # lost green time ratio
        loss = [np.abs(i / self.dis_diff_opt['idea_dis']) for i in dis_reals_to_ideas]

        offset_ratio = [i / 2 for i in loss]    # offset green time ratio

        phase_diff = []
        for i in range(len(id_reals_to_ideas)):
            if id_reals_to_ideas[i] % 2 == 0:
                phase_diff.append(0.5 - 0.5*ratio[i])
            else:
                phase_diff.append(1 - 0.5*ratio[i])

        up_ratio = []
        down_ratio = []
        for i in range(len(ratio)):
            if pos_reals_to_ideas[i] == 1:
                up_ratio.append(0.5*ratio[i] - offset_ratio[i])
                down_ratio.append(0.5*ratio[i] + offset_ratio[i])
            else:
                up_ratio.append(0.5*ratio[i] + offset_ratio[i])
                down_ratio.append(0.5*ratio[i] - offset_ratio[i])

        width = min(up_ratio) + min(down_ratio)

        idx = ['idea junction id', 'position', 'green time ratio', 'phase diffrernce', 'lost ratio', 'offset ratio', 
        'ratio on the center line', 'ratio down the center line']

        pos_char = []
        for i in pos_reals_to_ideas:
            if i == 0:
                pos_char.append('left')
            else:
                pos_char.append('right')

        output_table = [id_reals_to_ideas, pos_char, ratio, phase_diff, loss, offset_ratio, up_ratio, down_ratio]
        df_output = pd.DataFrame(output_table, columns=self.index_list, index=idx)

        return width, df_output

# testcase
test = [0, 300, 250, 380, 450, 300, 210]
ratio = [0.5, 0.55, 0.6, 0.6, 0.5, 0.7, 0.55]
line = LineCoordination(test)
opt_cycle, diff, diff_opt = line.findOptimal()
width, output = line.getOptimalInfo(ratio)
print("cycle: {}".format(opt_cycle))
print(diff)
print(diff_opt)
print("Band width: {}".format(width))
print(output)

cycle: 45.0
     idea_dis    0      1      2      3      4      5      6      b       b/a  \
0       240.0  0.0   60.0   70.0  210.0  180.0    0.0  210.0  110.0  0.458333   
1       241.0  0.0   59.0   68.0  207.0  175.0  234.0  203.0  107.0  0.443983   
2       242.0  0.0   58.0   66.0  204.0  170.0  228.0  196.0  104.0  0.429752   
3       243.0  0.0   57.0   64.0  201.0  165.0  222.0  189.0  101.0  0.415638   
4       244.0  0.0   56.0   62.0  198.0  160.0  216.0  182.0   98.0  0.401639   
..        ...  ...    ...    ...    ...    ...    ...    ...    ...       ...   
476     716.0  0.0  300.0  550.0  214.0  664.0  248.0  458.0  214.0  0.298883   
477     717.0  0.0  300.0  550.0  213.0  663.0  246.0  456.0  213.0  0.297071   
478     718.0  0.0  300.0  550.0  212.0  662.0  244.0  454.0  212.0  0.295265   
479     719.0  0.0  300.0  550.0  211.0  661.0  242.0  452.0  211.0  0.293463   
480     720.0  0.0  300.0  550.0  210.0  660.0  240.0  450.0  210.0  0.291667   

     b_index  


# 相对基准交叉口协调

In [16]:
dis_relative_horizontal = []
dis_relative_vertical = []

for i in dis_matrix[0]:
    dis_relative_horizontal.append(i[0])
for i in dis_matrix:
    dis_relative_vertical.append(i[0][1])

print(dis_relative_horizontal)
print(dis_relative_vertical)

line_horizontal = LineCoordination(dis_relative_horizontal)
line_vertical = LineCoordination(dis_relative_vertical)

opt_cycle_horizontal, diff_horizontal, diff_opt_horizontal = line_horizontal.findOptimal()
opt_cycle_vertical, diff_vertical, diff_opt_vertical = line_vertical.findOptimal()

print("Optimal cycle: ")
print("horizontal: {}, vertical: {}".format(opt_cycle_horizontal, opt_cycle_vertical))
print(diff_opt_horizontal)
print(diff_opt_vertical)

[0, 250, 540, 180]
[0, 300, 280, 320]
Optimal cycle: 
horizontal: 40.5, vertical: 50.0
idea_dis    243.000000
0             0.000000
1             7.000000
2            61.000000
3           241.000000
b           180.000000
b/a           0.740741
b_index       2.000000
Name: 0, dtype: float64
idea_dis    300.000000
0             0.000000
1             0.000000
2           280.000000
3             0.000000
b           280.000000
b/a           0.933333
b_index       2.000000
Name: 0, dtype: float64
